In [1]:
cd /content/drive/MyDrive/multi/202206/0614

/content/drive/MyDrive/multi/202206/0614


In [2]:
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [806 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRele

In [3]:
!pip install kss
!pip install git+https://github.com/ssut/py-hanspell.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.4 MB 1.2 MB/s 
     |████████████████████████████████| 175 kB 56.6 MB/s 
  Created wheel for kss: filename=kss-3.4.2-py3-none-any.whl size=42448069 sha256=f26c94fe4f8f3333bdf01d7755084a52901433d9e7f5a315c9637a8cdd1ba97d
  Stored in directory: /root/.cache/pip/wheels/ef/22/aa/6399b60516a067ec97fa6599fb2d472aeb25e3f9ee6dae3224
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=720aa80b365d1d1328893da6684fbeec19ca29ef6821f2c43f89201a28595802
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built kss emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-q838h08s
  Running command git clone -q https://github.com/ssut/py-hanspell

In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 11.9 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
     |████████████████████████████████| 596 kB 65.7 MB/s 
     |████████████████████████████████| 6.6 MB 63.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 28.4 MB/s 
     |████████████████████████████████| 140 kB 71.5 MB/s 
     |████████████████████████████████| 419 kB 70.5 MB/s 
     |████████████████████████████████| 1.1 MB 59.9 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 271 kB 60.2 MB/s 
     |████████████████████████████████| 144 kB 55.1 MB/s 


In [18]:
# ====================== Library ===========================
from unittest.util import _MAX_LENGTH
import pandas as pd
import re
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from transformers import PreTrainedTokenizerFast,GPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Mecab

# ==========================================================

# ====================Global Variable=======================
data = pd.read_csv('./data/2.Textranked.csv')

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPT_tok = PreTrainedTokenizerFast.from_pretrained("./models/koGPT2_tokenizer", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')
GPT = GPT2LMHeadModel.from_pretrained('./models/koGPT2_finetuned')

Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<mask>'
PAD = '<pad>'
# ============================================================

# ==========================Similarity========================
def get_df_token(data):
    tagger = Mecab()
    q_pos = []
    for i in range(len(data)):
        q_pos.append(tagger.pos(data['Question'][i]))
    data['tokens'] = q_pos
    pos = ["NNG","NNP","VV","VA"]
    data['tokens'] = data['tokens'].apply(lambda x: [t for (t, p) in x if p in pos])
    return data

def get_ques_token(question):
    tagger = Mecab()
    ques_token = tagger.pos(question)
    ques_token = ques_remove(ques_token)
    return ques_token

def ques_remove(question) :
    pos = ["NNG","NNP", "VV","VA"]
    ques_token = [[t for (t, p) in question if p in pos]]
    return ques_token

def jaccard_similarity(data, question):

    union = set(data).union(set(question))
    intersection = set(data).intersection(set(question))
    jaccard_sim = len(intersection) / len(union)  

    return jaccard_sim

def jaccard_high(data, question, num):
    # data: 데이터프레임, question: 입력한 텍스트(질문), num: 자카드 유사도 상위 갯수
    data['jaccard_similarity'] = data['tokens'].apply(lambda x: jaccard_similarity(x, question[0]))
    return data[['Question', 'Answer', 'tokens', 'jaccard_similarity']].sort_values(['jaccard_similarity'], ascending=False)[:num]

def tokenized_output(tokens):
    return tokens

def cos_similarity(data, question):
    tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                                       tokenizer=tokenized_output,
                                       preprocessor=tokenized_output,
                                       token_pattern=None)
    
    tfidf_data = tfidf_vectorizer.fit_transform(data['tokens'])
    tfidf_question = tfidf_vectorizer.transform(question)

    data['cosine_similarity'] = cosine_similarity(tfidf_data, tfidf_question).reshape(-1, )

    return data[['Question', 'Answer', 'tokens', 'jaccard_similarity', 'cosine_similarity']].sort_values(['cosine_similarity'], ascending=False)
# =======================================================================

# =================================Seq2Seq===============================
class QNA:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1:"EOS"}
        self.n_words = 2
        self.stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])

    def addSentence(self, sentence):
        for word in sentence.split('.'):
            self.addWord(word)

    def addWord(self, sen):
        tagger = Mecab()
        for word in [ val for (val, nn) in tagger.pos(sen) if (val, nn) not in self.stopword and nn in ('NNG', 'NNP', 'VV', 'VA')]:
            if word not in self.word2index:             
                self.word2index[word] = self.n_words
                self.word2count[word] = 1
                self.index2word[self.n_words] = word
                self.n_words += 1
            else:
              self.word2count[word] += 1

def normalizeString(s):
    s = re.sub(r"([.!?])", r" ", s)
    s = re.sub(r"[^가-힣.!?]+", r" ", s)
    return s

def readchat(question, answer, reverse=False):
    print("Reading lines...")

    lines = open('/content/drive/MyDrive/multi/202206/0614/data/2.Textranked.csv', encoding='utf-8').read().strip().split('\n')[1:]
    
    pairs = [[normalizeString(s) for s in l.split(',')] for l in lines]

    if reverse:
        paris = [list(reversed(p)) for p in pairs]
        input_q = QNA(answer)
        output_a = QNA(question)
    else:
        input_q = QNA(question)
        output_a = QNA(answer)
    
    return input_q, output_a, pairs

def filterPair(p):
    return len(p[0].split(' ')) < 2000 and len(p[1].split(' ')) < 2000

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def prepareData(question, answer, reverse=False):
    input_q, output_a, pairs = readchat(question, answer, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    for pair in pairs:
        input_q.addSentence(pair[0])
        output_a.addSentence(pair[1])
    print("Counted words:")
    print(input_q.name, input_q.n_words)
    print(output_a.name, output_a.n_words)
    return input_q, output_a, pairs

input_q, output_a, pairs = prepareData('question', 'answer', True)

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=DEVICE)

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=2000):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=DEVICE)
              
def indexesFromSentence(QNA, sentence):
    tagger = Mecab()
    word2index_list = []
    stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])
    for sen in sentence.split('.'):
        for word in [ val for (val, nn) in tagger.pos(sen) if (val, nn) not in stopword and nn in ('NNG', 'NNP', 'VV', 'VA')]:
            word2index_list.append(QNA.word2index[word])
    return word2index_list

def tensorFromSentence(QNA, sentence):
    EOS_token = 1
    indexes = indexesFromSentence(QNA, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=DEVICE).view(-1, 1)

def evaluate(encoder, decoder, sentence, max_length=2000):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_q, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=DEVICE)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]
        SOS_token = 0
        decoder_input = torch.tensor([[SOS_token]], device=DEVICE)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            EOS_token = 1
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_a.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

# ========================================================================================

def make_answer(data):
    q = input('Question:')
    q_token = get_ques_token(q)

    data = pd.read_csv('./data/2.Textranked.csv')
    d_token = get_df_token(data)

    result = jaccard_high(data, q_token, 100)
    result = cos_similarity(result, q_token)

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    Q_TKN = "<usr>"
    A_TKN = "<sys>"
    BOS = '</s>'
    EOS = '</s>'
    MASK = '<mask>'
    PAD = '<pad>'

    if result['cosine_similarity'].iloc[0] >= 0.6:
        return result['Answer'].iloc[0]
    else:
        if len(q) < 200:
            SOS_token = 0
            EOS_token = 1
            MAX_LENGTH = 2000
            tagger = Mecab()

            input_q, output_a, pairs = prepareData('question', 'answer', True)
            encoder = EncoderRNN(12518, 256).to(DEVICE)
            encoder.load_state_dict(torch.load('./models/Seq2Seq_mecab_encoder1.pth'))
            decoder = AttnDecoderRNN(256, 7755, dropout_p=0.1).to(DEVICE)
            decoder.load_state_dict(torch.load('./models/Seq2Seq_mecab_attn_decoder1.pth'))
            print("from Seq2seq:")
            evaluate(encoder, decoder, q)

        GPT_tok = PreTrainedTokenizerFast.from_pretrained("./models/koGPT2_tokenizer", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')
        GPT = GPT2LMHeadModel.from_pretrained('./models/koGPT2_finetuned')
        GPT.to(DEVICE)
        with torch.no_grad():
            q = q.strip()
            a = ""
            while 1:
                input_ids = torch.LongTensor(GPT_tok.encode(Q_TKN + q + A_TKN + a)).unsqueeze(dim=0).to(DEVICE)
                pred = GPT(input_ids)
                pred = pred.logits.to('cpu')
                gen = GPT_tok.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
                if gen == EOS:
                    break
                a += gen.replace("▁", " ")
            print("Answer > {}".format(a.strip()))

Reading lines...
Read 18165 sentence pairs
Counted words:
answer 12518
question 7755


In [7]:
df = pd.read_csv('/content/drive/MyDrive/multi/202206/0614/data/2.Textranked.csv')
df

,Question,Answer
0,정말 어려운 상황에서 너무나도 힘들어서 질문을 드립니다. 아버지가 제 명의로 사업자...,차량의 경우 현재 점유자인 아버지를 피고로 하여 차량 인도 청구소송 및 차량 점유 ...
1,년 전에 대포 통장이로 인해 피해를 봐서 재판 결과가 불기소 유통 지을 받았거든요....,소장 내용에 대한 검토가 필요하므로 소장을 지참하고 상담을 받으시기 바랍니다
2,제 계좌로 입금을 했다던가 그런부분 전혀없이 그 형의 계좌로 그 형의 공인인증서를 ...,별도의 구체적 상담이 필요하나 실제 채무가 없음에도 공증을 가지고 채권자가 강제집행...
3,채권시효년 소멸로 본다면 어떤 대응이 가능할지 여쭤봅니다 지인에게 연대보증 해줬는데...,별도의 구체적 상담이 필요하나 이행 권고 결정은 기판력의 제한이 없으므로 소멸시효 ...
4,그 와중에 피의자와 어떻게 연락이 닿게 되어서 갚겠다는 의사를 보였고 정말 적은 ...,가해자가 위 사건으로 구속 기소되었다면 가해자를 피고로 하여 민사소송 및 강제집행으...
...,...,...
18160,제가 작년 월쯤 승소 판결 받은 후에 압류 절차를 진행을 해야 하는데 혼자 스스로 ...,여쭤보신 것은 통장 압류 절차인 것으로 보입니다 통장 압류의 법률적 용어는 예금채...
18161,민사재판 패소 후 현재 등본은 부모님댁으로 되어있습니다,민사소송 패소 후 부모님 집에 압류가 들어올 수 있는지와 관련하여 질문하셨습니다.민...
18162,이에 원고인에게 원룸계약서 제출을 요구합니다 라고 썼거든요.거기보니 원고측에서 이미...,따라서 각 월 차임 채권은 년에 발생한 경우 년이 경과한 년 사이에 이미 시효...
18163,오늘 피고 측에서 답변서를 받고 변론 기일이 정해졌는데 이제 보니 제가 작성한 소...,민사소송 소장에 소송비용 청구를 빠드린 경우와 관련하여 질 무하셨습니다. 민사소송에...


In [19]:
make_answer(df)


Question:여자친구가 제 차를 가지고 도망갔습니다. 며칠전까지 같이 있었습니다. 처벌 가능할까요?
Reading lines...
Read 18165 sentence pairs
Counted words:
answer 12518
question 7755
from Seq2seq:
Answer > 혹시 사안과 관련해 좀 더 구체적인 상담을 원하시면  상담 요청을 통해 확실하게 정리를 하실 수 있습니다  변호인 선임 여부에 대한 판단이 잘 서지 않을 때 또는 그냥 단순히 사안에 대해 좀 더 구체적인 법률상담이 필요할 때 언제든지 모두 환영이니 엑스퍼트 버전 접속  엑스퍼트 모바일 버전 접속
